# Tutorial: applying strainFlye to the SheepGut dataset

This tutorial will walk you through some of the analyses that strainFlye can perform.

Here we will be using the same SheepGut dataset that is used in our paper, but feel free to follow along with another dataset.

The pipeline takes as input two primary types of data:

1. A __set of reads__ (in FASTA / FASTQ format).

2. A __set of contigs__ (in FASTA format) assembled from these reads.

## Details about the inputs

**Regarding reads:** We designed strainFlye in the context of PacBio Circular Consensus Sequencing (CCS) "HiFi" reads ([Wenger & Peluso _et al._, 2019](https://www.nature.com/articles/s41587-019-0217-9)). However, in theory it should still work with other reasonably long and accurate reads.

**Regarding contigs:** We don't impose any restriction on the assembler you use to construct these. We designed strainFlye in the context of [metaFlye](https://github.com/fenderglass/Flye) ([Kolmogorov _et al._, 2020](https://www.nature.com/articles/s41587-019-0217-9)) output, but it should work with the outputs of other HiFi assemblers.

## The SheepGut dataset

Please see the paper's "Data access" section for details about acquiring both of these types of data for the SheepGut dataset.

Note that the "contigs" we use for the SheepGut datatset really correspond to edge sequences in the `assembly_graph.gfa` file produced by metaFlye -- in general, these may be slightly different from the file of contigs / scaffolds in the `assembly.fasta` file produced by metaFlye: see [Flye's manual](https://github.com/fenderglass/Flye/blob/flye/docs/USAGE.md#output) for more information. (You could use either type of sequence with strainFlye, although I personally recommend using edges -- it's useful to have context about where exactly in the assembly graph a sequence is, and things like gaps in scaffolds will cause strainFlye to complain.)

## Introduction

Let's take care of a few things before the tutorial starts.

### Installing strainFlye

Before following along with this tutorial, we assume that you have already installed strainFlye (and have activated the corresponding conda environment). Please see [strainFlye's README](https://github.com/fedarko/strainFlye) for installation instructions.

### What commands are available through strainFlye?

In [1]:
!strainFlye

Usage: strainFlye [OPTIONS] COMMAND [ARGS]...

  Pipeline for the analysis of rare mutations in metagenomes.

  Please consult https://github.com/fedarko/strainFlye if you have any
  questions, comments, etc. about strainFlye. Thank you for using this tool!

Options:
  -h, --help  Show this message and exit.

Commands:
  align   Align reads to contigs, and filter the resulting alignment.
  call    [+] Call mutations in contigs naïvely & compute diversity indices.
  fdr     [+] Estimate and fix FDRs for contigs' naïve mutation calls.
  spot    [+] Identify putative mutational hotspots or coldspots in contigs.
  smooth  [+] Create and assemble smoothed and virtual reads.
  link    [+] Create link graphs in order to show mutations' co-occurrences.
  utils   [+] Various utility commands provided with strainFlye.


### Importing and configuring some utilities

You shouldn't need to actually do any programming to use strainFlye, but we'll be using Python to perform some plotting and other analyses throughout this tutorial—we'll import some useful packages here to reduce clutter in this notebook.

(If you prefer, you could of course use another language instead of Python.)

In [9]:
import os
import time
import skbio
import pandas as pd
from math import floor
from matplotlib import pyplot

# Make our plots look pretty
# (this is the same style we use throughout most of the paper's figures)
pyplot.style.use("ggplot")

## 0. Convert the assembly graph GFA file to a FASTA file of contigs

**You can skip this step if:** you already have a FASTA file describing contigs in your assembly graph.

Our assembly graph (the GFA file) contains the sequences of the contigs that we will use in many downstream analyses, but we'll need to have a FASTA file that just describes these contigs' sequences (independent of the assembly graph topology).

There are some [bash one-liners](https://www.biostars.org/p/169516/#169530) you can use to convert a GFA 1 file to a FASTA file, but strainFlye also provides a utility command (`strainFlye utils gfa-to-fasta`) to do this for you. We'll use this here. (Our solution may be a bit slower than a bash one-liner, but it performs some useful sanity checking on the GFA file.)

In [46]:
!strainFlye utils gfa-to-fasta \
    --graph /Poppy/mfedarko/misc-data/sheepgut_flye_big_2.8_graph.gfa \
    --output-fasta /Poppy/mfedarko/sftests/tutorial-output/sheepgut_contigs.fasta

--------
strainFlye utils gfa-to-fasta @ 0.00 sec: Starting...
Input GFA file: /Poppy/mfedarko/misc-data/sheepgut_flye_big_2.8_graph.gfa
Output FASTA file: /Poppy/mfedarko/sftests/tutorial-output/sheepgut_contigs.fasta
--------
strainFlye utils gfa-to-fasta @ 14.97 sec: Done.
Output FASTA file contains 78,793 sequences.


## 1. Align reads to contigs; filter the resulting alignment

**You can skip this step if:** you already have a BAM file representing an alignment of reads to contigs, and this BAM file does not contain secondary alignments / partially-mapped reads / overlapping supplementary alignments (these all may cause problems in downstream analyses).

We'll need to align reads back to these contigs. The resulting alignment, and/or the mutations that we call from it, will be used in pretty much all downstream steps—so it's important to make sure that it is of good quality!

The `strainFlye align` command uses minimap2 to perform alignment, and then does some extra filtering on the resulting alignment.

Note that this command, in particular, may take a while to run. Sequence alignment is computationally expensive! On our cluster, `strainFlye align` ran on the full SheepGut dataset in 62,941.21 seconds (aka about 17.5 hours).

In [5]:
!strainFlye align

Usage: strainFlye align [OPTIONS] READS...

  Align reads to contigs, and filter the resulting alignment.

  Files of reads should be in the FASTA or FASTQ formats; GZIP'd files are
  allowed.

  This command involves multiple steps, including:

    1) Align reads to contigs (using minimap2) to generate a SAM file
    2) Convert this SAM file to a sorted and indexed BAM file
    3) Filter overlapping supplementary alignments (OSAs) from this BAM file
    4) Filter partially-mapped reads from this BAM file

  Note that we only sort the alignment file once, although we do re-index it
  after the two filtering steps. This decision is motivated by
  https://www.biostars.org/p/131333/#131335.

Options:
  -c, --contigs PATH              FASTA file of contigs to which reads will be
                                  aligned.  [required]
  -g, --graph PATH                GFA 1-formatted file describing an assembly
                                  graph of the contigs. This is used in the
     

In [4]:
!strainFlye align \
    # We can use the FASTA file we just generated above.
    --contigs /Poppy/mfedarko/sftests/tutorial-output/sheepgut_contigs.fasta \
    --graph /Poppy/mfedarko/misc-data/sheepgut_flye_big_2.8_graph.gfa \
    --output-dir /Poppy/mfedarko/sftests/tutorial-output/alignment \
    # Reads file(s) are specified here, after all of the other parameters:
    /Poppy/mkolmogo/sheep_meta/data/sheep_poop_CCS_dedup.fastq.gz \
    /Poppy/mkolmogo/sheep_meta/data/ccs_sequel_II/*.fasta.gz

This generates a BAM file (`final.bam`) and BAM index file (`final.bam.bai`) in the specified output directory.

We can use this BAM file for many analyses downstream—the first of these will be mutation calling.

## 1.5. Optional: filter the FASTA file in order to focus on "long" contigs

We just aligned our dataset's reads against *all* contigs in the assembly graph. This is standard practice (see, e.g., [this tutorial](https://astrobiomike.github.io/genomics/metagen_anvio#mapping-our-reads-to-the-assembly-they-built)); aligning reads against all contigs probably yields a more accurate alignment than just aligning reads against a subset of these contigs (although proving if this is "best practice" or not is a challenging question, and one that I will sidestep right now).

However, now that we have this alignment, we don't necessarily need to perform mutation calling, phasing, etc. on all contigs (although, if you want to, we could!). To speed up the rest of this tutorial, we will focus solely on the "long" contigs in this dataset: here, we will define a contig as "long" if its length is **at least 1 Mbp**. In theory, these long contigs represent putative metagenome-assembled genomes (MAGs).

Of course, if you prefer, you could apply more sophisticated criteria to pick which contigs to focus on—maybe you'd like to also focus on contigs with high coverages, or maybe on contigs with good [CheckM](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4484387/) completeness or contamination values. Or maybe you'd like to keep considering all contigs in the full dataset! Your decision should depend on your goals, and your dataset.

In any case, how do we "focus on" certain contigs? **We can filter our FASTA file to a subset of contigs present in the full dataset**, and use this filtered FASTA file for all downstream analyses. As an example of this, we will use Python (in particular, with the [scikit-bio](http://scikit-bio.org/) library) to filter our FASTA file to all long contigs.

In [15]:
# Produce a filtered FASTA file containing only contigs >= 1 Mbp long
# (This uses scikit-bio; see http://scikit-bio.org/ for more details.)

input_contigs_fp = "/Poppy/mfedarko/sftests/tutorial-output/sheepgut_contigs.fasta"
output_contigs_fp = "/Poppy/mfedarko/sftests/tutorial-output/sheepgut_contigs_atleast_1Mbp.fasta"
len_threshold = 1000000

t0 = time.time()
num_long_contigs = 0
with open(output_contigs_fp, "w") as of:
    for contig in skbio.io.read(input_contigs_fp, format="fasta", constructor=skbio.DNA):
        if len(contig) >= len_threshold:
            skbio.io.write(contig, format="fasta", into=of)
            num_long_contigs += 1
t1 = time.time()

print(f"Found {num_long_contigs:,} contigs with lengths \u2265 {len_threshold:,} bp. Took {t1 - t0:,.2f} sec.")

Found 468 contigs with lengths ≥ 1,000,000 bp. Took 12.30 sec.


The remainder of this tutorial will focus on these 468 long contigs.

In any case, now we can get on to some more interesting stuff!

## 2. Perform naïve mutation calling, then estimate and fix mutation calls' FDRs

**You can skip this step if:** You already have a BCF file describing single-nucleotide, non-multi-allelic mutations in your contigs.

The analyses downstream of this step (hotspot/coldspot identification, phasing) take as input a set of identified single-nucleotide mutations (or, if you prefer to use different terminology, "called variants", "called SNVs", ...) in which we have some confidence. How does strainFlye identify these mutations?

There are a few steps (as our paper describes). First, we will **naïvely call mutations** using a simple threshold-based method (referred to as "NaiveFreq" in the paper). We'll then **estimate the false-discovery rates of the mutations called for each contig** using the target-decoy approach, and then adjust the called mutations to **fix the false-discovery rates of these mutation calls** below a specified threshold.

### 2.1. $p$-mutations and $r$-mutations?

So, our first step will be performing this simple threshold-based calling. What do we mean by "threshold" here?

strainFlye supports calling two basic types of mutations: $p$-mutations and $r$-mutations. The docs explain the difference between these two types best:

In [16]:
!strainFlye call

Usage: strainFlye call [OPTIONS] COMMAND [ARGS]...

  [+] Call mutations in contigs naïvely & compute diversity indices.

  Consider a position "pos" in a contig. A given read with a (mis)match
  operation at "pos" must have one of four nucleotides (A, C, G, T) aligned to
  pos. We represent these nucleotides' counts at pos as follows:

      N1 = # reads of the most-common aligned nucleotide at pos,
      N2 = # reads of the second-most-common aligned nucleotide at pos,
      N3 = # reads of the third-most-common aligned nucleotide at pos,
      N4 = # reads of the fourth-most-common aligned nucleotide at pos.

  (We break ties arbitrarily.)

  strainFlye supports two types of naïve mutation calling based on these
  counts: p-mutations and r-mutations. These are described below.

  p-mutations (naïve percentage-based mutation calling)
  -----------------------------------------------------

  This takes as input some percentage p in the range (0%, 50%]. Define
  freq(pos) = N2 / (N1 +

### 2.2. Understanding these (sub)commands

First off, note that `strainFlye call` doesn't do anything besides show help info if you run it by itself. This is because, unlike `strainFlye align`, `strainFlye call` has two subcommands: `p-mutation` and `r-mutation`. Which of these you use will depend on how you want to naïvely call mutations. You can invoke one of these subcommands by writing out the full chain of commands: for example, `strainFlye call p-mutation`.

#### 2.2.1. Input and output

Probably the most important parameter at this step is the *minimum threshold*. Both of these subcommands, `strainFlye call p-mutation` and `strainFlye call r-mutation`, take as input a minimum version of their corresponding threshold (either `--min-p` or `--min-r`).

These commands each output:

1. A __BCF (binary [variant call format](https://samtools.github.io/hts-specs/VCFv4.3.pdf)) file__ describing all mutations called naïvely across the contigs, based on the minimum $p$ or $r$ threshold set (`--min-p` or `--min-r`).

2. A __TSV ([tab separated values](https://en.wikipedia.org/wiki/Tab-separated_values)) file__ describing the contigs' computed diversity indices, for various values of $p$ or $r$ (controllable using the `--div-index-p-list` or `--div-index-r-list` parameters). Long story short, diversity indices indicate how many of a contig's (sufficiently-covered) positions have called mutations: in general, higher diversity indices mean more mutations.

#### 2.2.2. Interpreting the output

The default minimum value of $p$ (or $r$) used in these commands is fairly low. As you might expect, using such a low threshold for calling a position as a mutation will yield many false positives: we will almost certainly identify many real mutations, but also many "false" mutations that occur as the result of sequencing errors, alignment errors, etc. Viewed another way, the __false discovery rate (FDR)__ (defined as the ratio of false positives to total true + false positives) of the mutation calls generated at this step will probably be unacceptably high.

So, after we run this command, we'll use strainFlye's FDR estimation and fixing functionality to try to address this problem. This will involve adjusting the "minimum" value of $p$ or $r$ used for each contig to reduce the FDR as needed. To emphasize the point, **you probably shouldn't use the naïvely called mutations at this step for downstream analyses without first estimating and fixing FDRs** (unless you know what you're doing).

### 2.3. Naïvely call $p$-mutations ($p = 0.15\%$) and compute diversity indices for various values of $p$

Now that we know what we're doing, we're ready to call mutations and compute diversity indices! We'll do $p$-mutation calling at a minimum $p$ of $0.15\%$, which matches what we used for Figure 2 in the paper. The default diversity index values of $p$ (ranging from $0.5\%$ to $50\%$) should be good for us.

Note that this command will take a while—we need to check each position in the alignment for each of the input contigs. We will use the `--verbose` flag to display some extra information about the status of each contig while this command is running, to make the wait more tolerable.

In [17]:
!strainFlye call p-mutation

Usage: strainFlye call p-mutation [OPTIONS]

  Call p-mutations and compute diversity indices.

  The primary parameter for this command is the lower bound of p, defined by
  --min-p. The BCF output will include "mutations" for all positions that pass
  this (likely very low) threshold, but this BCF should be adjusted using the
  utilities contained in the "strainFlye fdr" module.

Options:
  -c, --contigs PATH              FASTA file of contigs in which to naïvely
                                  call mutations. All contigs in this FASTA
                                  file should also be contained in the BAM
                                  file; however, if some contigs in the BAM
                                  file are not included in this FASTA file, we
                                  won't perform any calling on these absent
                                  contigs.  [required]
  -b, --bam PATH                  Sorted and indexed BAM file representing an
               

In [ ]:
!strainFlye call p-mutation \
    --contigs /Poppy/mfedarko/sftests/tutorial-output/sheepgut_contigs_atleast_1Mbp.fasta \
    --bam /Poppy/mfedarko/sftests/tutorial-output/alignment/final.bam \
    --min-p 15 \
    --output-dir /Poppy/mfedarko/sftests/tutorial-output/call-p15 \
    --verbose

### 2.4. Analyzing diversity indices: the search for a decoy contig

We now have both our initial mutation calls (which, as we've discussed, probably have a high FDR) and information about our contigs' diversity indices. We will use the __target-decoy approach__ to attempt to estimate and thus control the FDR of our mutation calls. This is done by the `strainFlye fdr estimate` and `strainFlye fdr fix` commands.

As discussed in our paper, we can select—out of one of our $C$ contigs—a __decoy contig__ (a.k.a. a decoy genome), and compute a mutation rate for it ($\text{rate}_{\text{decoy}}$). For each of the other $C - 1$ __target contigs__, we can estimate the FDR of identified mutations in this contig as $\dfrac{\text{rate}_{\text{decoy}}}{\text{rate}_{\text{target}}}$.

If you'd like, we could go through the diversity indices produced by `strainFlye call p-mutation` ourselves, in an attempt to select a reasonable-seeming decoy contig. **[This notebook](https://nbviewer.org/github/fedarko/strainFlye/blob/main/docs/AnalyzingDiversityIndices.ipynb)** demonstrates this sort of process. However, `strainFlye fdr estimate` can also do this sort of thing automatically; so, in order to not make this tutorial longer than it already is, we'll move on to FDR estimation.

### 2.5. Estimating FDRs using the target-decoy approach

The optional notebook discussed above shows that `edge_6104` is probably a good decoy contig, so we could if desired just pass it to `strainFlye fdr estimate` using that command's `-dc` or `--decoy-contig` option. However, to illustrate another option, we'll instead pass our diversity index TSV file to `strainFlye fdr estimate` and let it do the job of selecting a decoy contig. (Spoiler alert: it'll select `edge_6104` anyway.)

#### 2.5.1. Sidenote: strainFlye's algorithm for automatically selecting a decoy contig

If `strainFlye fdr estimate` is provided a diversity index file, it will automatically select a decoy contig using the following _ad hoc_ algorithm:

1\. Filter to all contigs whose lengths and average coverages meet the `--decoy-min-length` and `--decoy-min-average-coverage` thresholds, respectively. Define $C$ as the set of all contigs that meet these thresholds (so, $|C|$ indicates the number of contigs in $C$).

  - If $|C| = 0$, raise an error (the length and coverage thresholds should probably be lowered for this dataset).
  - If $|C| = 1$, select the lone contig in $C$ as the decoy.
  - If $|C| = 2$, move on to the next steps.


2\. Define $D$ as the set of all diversity index columns in the input file (representing computed diversity indices at different values of $p$ or $r$), where at least two of the $C$ passing contigs have a diversity index defined for this column.

  - If $|D| = 0$, raise an error.
  - Otherwise, move on to the next steps.


3\. For each column in $D$:

  - Compute the minimum ($\text{min}$) and maximum ($\text{max}$) diversity index in this column, only considering contigs in $C$.
  - In the strange case where $\text{min} = \text{max}$, ignore this column and move on to the next. (If all columns in $D$ have this problem, raise an error.)
  - Otherwise, assign each contig in $C$ a score for this column.
    - For each contig with a defined diversity index in this column, assign a score using [linear interpolation](https://en.wikipedia.org/wiki/Linear_interpolation). Higher scores indicate that a contig seems "more" diverse. If the diversity index of this contig in this column is defined as $x$, then this contig's score for this column, $\text{score}(x)$, is defined as $\text{score}(x) = \dfrac{x - \text{min}}{\text{max} - \text{min}}$.
    - For each contig with an undefined diversity index in this column, penalize this contig by assigning it a score of 1.
    
    
4\. Sum scores across all columns in $D$ for each contig in $C$. **Select the contig with the lowest score sum as the decoy contig**, breaking ties arbitrarily.


For more details about how this works, please see the source code for the [`autoselect_decoy()` function](https://github.com/fedarko/strainFlye/blob/main/strainflye/fdr_utils.py#L100).

In [6]:
!strainFlye fdr

Usage: strainFlye fdr [OPTIONS] COMMAND [ARGS]...

  [+] Estimate and fix FDRs for contigs' naïve mutation calls.

Options:
  -h, --help  Show this message and exit.

Commands:
  estimate  Estimate the FDRs of contigs' naïve mutation calls.
  fix       Fix contigs' naïve mutation calls' FDRs to an upper limit.


In [3]:
!strainFlye fdr estimate

Usage: strainFlye fdr estimate [OPTIONS]

  Estimate the FDRs of contigs' naïve mutation calls.

  We do this using the target-decoy approach (TDA). Given a set of C contigs,
  we select a "decoy contig" with relatively few called mutations. We then
  compute a mutation rate for this decoy contig, and use this mutation rate
  (along with the mutation rates of the other C - 1 "target" contigs) to
  estimate the FDRs of all of these target contigs' mutation calls.

  We can produce multiple FDR estimates for a single target contig's calls by
  varying the p or r threshold used (from the --min-p or --min-r threshold
  used to generate the input BCF file, up to the --high-p or --high-r
  threshold given here). Using this information (and information about the
  numbers of mutations called per megabase), we can plot an FDR curve for a
  given target contig's mutation calls.

Options:
  -c, --contigs PATH              FASTA file of contigs.  [required]
  --bam PATH                      Sorte

In [7]:
!strainFlye fdr estimate \
    --contigs /Poppy/mfedarko/sftests/tutorial-output/sheepgut_contigs_atleast_1Mbp.fasta \
    --bam /Poppy/mfedarko/sheepgut/main-workflow/output/fully-filtered-and-sorted-aln.bam \
    --bcf /Poppy/mfedarko/sftests/tutorial-output/call-p15/naive-calls.bcf \
    --diversity-indices /Poppy/mfedarko/sftests/tutorial-output/call-p15/diversity-indices.tsv \
    --decoy-contexts Full \
    --decoy-contexts CP2 \
    --decoy-contexts Tv \
    --decoy-contexts Nonsyn \
    --decoy-contexts Nonsense \
    --decoy-contexts CP2Tv \
    --decoy-contexts CP2Nonsyn \
    --decoy-contexts CP2Nonsense \
    --decoy-contexts TvNonsyn \
    --decoy-contexts TvNonsense \
    --decoy-contexts CP2TVNonsense \
    --output-dir /Poppy/mfedarko/sftests/tutorial-output/p15-fdr-info

--------
strainFlye fdr estimate @ 0.00 sec: Starting...
Input contig file: /Poppy/mfedarko/sftests/tutorial-output/sheepgut_contigs_atleast_1Mbp.fasta
Input BAM file: /Poppy/mfedarko/sheepgut/main-workflow/output/fully-filtered-and-sorted-aln.bam
Input BCF file: /Poppy/mfedarko/sftests/tutorial-output/call-p15/naive-calls.bcf
Input diversity indices file: /Poppy/mfedarko/sftests/tutorial-output/call-p15/diversity-indices.tsv
Input manually-set decoy contig: None
Input decoy contig context-dependent position / mutation type(s): ('Full', 'CP2', 'Tv', 'Nonsyn', 'Nonsense', 'CP2Tv', 'CP2Nonsyn', 'CP2Nonsense', 'TvNonsyn', 'TvNonsense', 'CP2TvNonsense')
Input high p threshold (only used if the BCF describes p-mutations): 500
Input high r threshold (only used if the BCF describes r-mutations): 100
Input min length of a potential decoy contig (only used if diversity indices are specified): 1000000
Input min average coverage of a potential decoy contig (only used if diversity indices are spec

Let's check on the TSV files that got written to the output directory. We should see one file for every decoy context, indicating the FDR estimates for each target contig for this context; and one lone "number of mutations per Mb" file, indicating the number of mutations per megabase for each target contig.

In general, we can plot these as FDR curves by using the FDR estimates as x-axis values and the "number of mutations per Mb" values as y-axis values.

In [8]:
!ls /Poppy/mfedarko/sftests/tutorial-output/p15-fdr-info/

fdr-CP2Nonsense.tsv    fdr-CP2Tv.tsv	 fdr-TvNonsense.tsv
fdr-CP2Nonsyn.tsv      fdr-Full.tsv	 fdr-TvNonsyn.tsv
fdr-CP2.tsv	       fdr-Nonsense.tsv  fdr-Tv.tsv
fdr-CP2TvNonsense.tsv  fdr-Nonsyn.tsv	 num-mutations-per-mb.tsv


### 2.6. Plotting FDR curves

**[This notebook](https://nbviewer.org/github/fedarko/strainFlye/blob/main/docs/PlottingFDRCurves.ipynb)** demonstrates how we can plot some or all of these FDR estimates for our target contigs.

### 2.7. Fixing mutation calls' FDRs to an upper limit of $1\%$

FDR estimates are nice, but the main thing we want is the ability to _fix_ these FDRs for each target contig's mutation calls to a specified upper limit.

Since we already have FDR curves showing how, as $p$ varies, the estimated FDR for each target contig varies, fixing the estimated FDR to an upper limit $F$ amounts to choosing a value of $p$ that yields a FDR ≤ $F$. The `strainFlye fdr fix` command takes care of this for us.

In [64]:
!strainFlye fdr fix

Usage: strainFlye fdr fix [OPTIONS]

  Fix contigs' naïve mutation calls' FDRs to an upper limit.

  This takes as input the estimated FDRs from "strainFlye fdr estimate" (if
  you used multiple decoy contexts, then you will need to choose which set of
  FDR estimates to use here) to guide us on how to fix the FDR for each
  contig. Note that mutations that passed the "high" p or r threshold
  specified for "strainFlye fdr estimate", and thus were not used for FDR
  estimation, will all be included in the output BCF file from this command;
  these mutations are considered "indisputable."

  We include indisputable mutations from the decoy contig and from all target
  contigs our output BCF file. We will only consider including non-
  indisputable mutations from the target contigs: the decision of which non-
  indisputable mutations will be included is based on the lowest p or r
  parameter for a target contig that yields an estimated FDR ≤ the fixed FDR
  given here.

Options:
  -b, --

In [30]:
!strainFlye fdr fix \
    --bcf /Poppy/mfedarko/sftests/tutorial-output/call-p15/naive-calls.bcf \
    --fdr-info /Poppy/mfedarko/sftests/tutorial-output/p15-fdr-info.tsv \
    --fdr 1 \
    --output-bcf /Poppy/mfedarko/sftests/tutorial-output/p15-fdr1pct.bcf

--------
strainFlye fdr fix @ 0.00 sec: Starting...
Input BCF file: /Poppy/mfedarko/sftests/tutorial-output/call-p15/naive-calls.bcf
Input FDR estimate file: /Poppy/mfedarko/sftests/tutorial-output/p15-fdr-info.tsv
Input FDR to fix mutation calls at: 1.0
Verbose?: No
Output BCF file with mutation calls at the fixed FDR: /Poppy/mfedarko/sftests/tutorial-output/p15-fdr1pct.bcf
--------
strainFlye fdr fix @ 0.00 sec: Loading and checking BCF and TSV files...
strainFlye fdr fix @ 14.93 sec: Looks good so far; decoy contig seems to be edge_6104.
strainFlye fdr fix @ 14.93 sec: Looks like the cutoff for "indisputable" mutations was p = 500.
strainFlye fdr fix @ 14.93 sec: All mutations passing this cutoff will be included in the output BCF file.
--------
strainFlye fdr fix @ 14.93 sec: Based on the FDR information, finding optimal values of p for each contig...
strainFlye fdr fix @ 14.95 sec: Done.
strainFlye fdr fix @ 14.95 sec: For 155 / 467 contigs, there exist values of p (at least, cons

It took us a few steps, but we have now generated a file (`p15-fdr1pct.bcf`) of $p$-mutation calls at a fixed (estimated) FDR of 1%.

Although our methodology has a few limitations (e.g. we don't support calling multi-allelic mutations yet), this BCF file can be used downstream for many types of analyses. In the next sections of the tutorial we'll demonstrate the additional commands supported by strainFlye, most of which make use of these mutation calls in some way.

## 3. Identifying hotspots and coldspots

So, now we can get to the fun part!

Given our called mutations, we may be interested in analyzing their locations in the contigs. Are there any particular regions, for example, where there are surprisingly many mutations? Are there any long regions where there are, surprisingly, no mutations?

We've kept strainFlye's functionality for identifying these types of regions fairly minimal at the moment. Here we'll demonstrate identifying basic hotspots and coldspots using `strainFlye spot`'s commands.

In [31]:
!strainFlye spot

Usage: strainFlye spot [OPTIONS] COMMAND [ARGS]...

  [+] Identify putative mutational hotspots or coldspots in contigs.

  Many methods exist for identifying these sorts of hotspots or coldspots; so,
  strainFlye's implementations of these methods are intended mostly as a quick
  proof-of-concept for replicating the results shown in our paper, and are not
  extremely robust quite yet.

Options:
  -h, --help  Show this message and exit.

Commands:
  hot-features  Identify hotspot features (for example, genes).
  cold-gaps     Identify long coldspot "gaps" without any mutations.


### 3.1. Identifying hotspot features

In [32]:
!strainFlye spot hot-features

Usage: strainFlye spot hot-features [OPTIONS]

  Identify hotspot features (for example, genes).

  By "feature", we refer to a single continuous region within a contig, as
  described in the file given for --features. These regions could describe
  anything: predicted protein-coding genes, introns or exons, intergenic
  regions of interest, etc. For now, we treat each feature independently (e.g.
  we don't lump together exons from the same "Parent" gene; each feature is
  considered separately as a potential "hotspot").

  You can configure whether or not we classify a feature as a hotspot by
  adjusting the --min-num-mutations and --min-perc-mutations parameters; at
  least one of these parameters must be specified. If both parameters are
  specified, then both checks (number of mutations in a feature, and
  percentage of mutations in a feature) will need to pass in order for us to
  label a feature as a hotspot.

Options:
  -b, --bcf PATH                  Indexed BCF file describing

#### 3.1.1. A note about "features"

Although we should be familiar with the FASTA and BCF input files by this point, the `-f` / `--features` input (a GFF3 file) may be surprising. strainFlye leaves the task of creating this file up to the user.

Predicted genes' coordinates are probably the most obvious type of "feature" for which we could look for hotspots. If you don't have gene predictions for your contigs yet, [Prodigal](https://github.com/hyattpd/Prodigal) is good (and should have been installed along with strainFlye, since other parts of strainFlye make use of it internally). Here, we'll use it to predict protein-coding genes across all contigs.

It's important to note that Prodigal does not predict eukaryotic genes (i.e. genes that are split up into introns and exons). These genes will thus not be a perfect representation of all protein-coding genes in all contigs in the dataset, since we know that this sample does contain at least some eukaryotic genomes. (However, if you use another tool for predicting eukaryotic genes that produces GFF3 output, then these should also be usable as "features" for this command.)

In [ ]:
# See https://github.com/hyattpd/Prodigal/wiki/cheat-sheet for details about these options.
#
# Note that, for the paper, I ran Prodigal in "normal" mode on certain contigs individually
# (https://github.com/fedarko/sheepgut/blob/main/inspect-seqs/prodigal.py), but here we just
# run Prodigal in "anonymous" mode on all contigs at once. The results should be fairly similar.
!prodigal \
    -i /Poppy/mfedarko/sftests/tutorial-output/sheepgut_contigs_atleast_1Mbp.fasta \
    -f gff \
    -c \
    -p meta \
    -o /Poppy/mfedarko/sftests/tutorial-output/prodigal_anonymous.gff

#### 3.1.2. Running the command to identify hotspot features (hotspot predicted genes)

Now that we have our gene predictions, let's move see if any of them have a lot of mutations. (Based on our findings in the paper, we know that these sorts of hotspots do exist in this dataset.)

`strainFlye spot hot-features` supports two types of basic thresholds for labelling a feature as a hotspot, `--min-num-mutations` and `--min-perc-mutations`. We'll use both here, and label a feature a "hotspot" if it meets both of the following criteria:

1. It contains at least 5 mutations.
2. At least 2% of its positions have mutations.

In [2]:
!strainFlye spot hot-features \
    --bcf /Poppy/mfedarko/sftests/tutorial-output/p15-fdr1pct.bcf \
    --features /Poppy/mfedarko/sftests/tutorial-output/prodigal_anonymous.gff \
    --min-num-mutations 5 \
    --min-perc-mutations 2 \
    --output-hotspots /Poppy/mfedarko/sftests/tutorial-output/hotspot-features-n5p2.tsv

--------
strainFlye spot hot-features @ 0.00 sec: Starting...
Input BCF file: /Poppy/mfedarko/sftests/tutorial-output/p15-fdr1pct.bcf
Input feature file: /Poppy/mfedarko/sftests/tutorial-output/prodigal_anonymous.gff
Input minimum number of mutations needed to call a feature a hotspot: 5
Input minimum % of mutations needed to call a feature a hotspot: 2.0
Output file describing hotspot features: /Poppy/mfedarko/sftests/tutorial-output/hotspot-features-n5p2.tsv
--------
strainFlye spot hot-features @ 0.00 sec: Loading and checking the BCF file...
strainFlye spot hot-features @ 15.32 sec: Looks good so far.
--------
strainFlye spot hot-features @ 15.32 sec: Going through features in the GFF3 file and identifying hotspot features...
strainFlye spot hot-features @ 121.92 sec: Identified 57,188 hotspot feature(s) across all 468 contigs in the BCF file.
--------
strainFlye spot hot-features @ 121.92 sec: Writing out this information to a TSV file...
strainFlye spot hot-features @ 122.09 sec:

The output of this command isn't anything special: it's a TSV file in which each row describes an identified hotspot feature. Let's load this file using `pandas.read_csv()` and get a brief sense of what it looks like:

In [5]:
hotspots = pd.read_csv("/Poppy/mfedarko/sftests/tutorial-output/hotspot-features-n5p2.tsv", sep="\t")
hotspots.head()

,Contig,FeatureID,FeatureStart_1IndexedInclusive,FeatureEnd_1IndexedInclusive,NumberMutatedPositions,PercentMutatedPositions
0,edge_8,8_765,865350,866369,32,3.14%
1,edge_8,8_781,873414,873626,5,2.35%
2,edge_8,8_787,878293,879429,48,4.22%
3,edge_8,8_788,879444,881552,101,4.79%
4,edge_8,8_789,881718,882956,42,3.39%


### 3.2. Identifying coldspot gaps

Similarly, strainFlye supports the identification of basic "coldspots"—here, defined as long gaps without any mutations. The main parameter is the minimum length needed to define a gap as a "coldspot." Let's test this out on the SheepGut dataset:

In [14]:
!strainFlye spot cold-gaps

Usage: strainFlye spot cold-gaps [OPTIONS]

  Identify long coldspot "gaps" without any mutations.

  To clarify, we define a "gap" of length L on a contig as a collection of
  continuous positions [N, N + 1, ... N + L - 2, N + L - 1] in which no
  positions are mutations (based on the input BCF file).

  If the --circular flag is provided, then we can loop around the contig from
  right to left; otherwise, the left and right sides of the contig are hard
  boundaries. To give an example of this, consider a 9-nucleotide contig that
  has mutations at positions 4 and 6:

                             Mut.    Mut.
                  1   2   3   4   5   6   7   8   9

  If --circular is provided, then this contig has two gaps: one gap of length
  1 (covering just position 5, between the two mutations), and another gap of
  length 6 (starting at position 7 and looping around to position 3: [7, 8, 9,
  1, 2, 3]).

  If --circular is not provided, then this contig has three gaps: [1, 2, 3],
  [

In [7]:
!strainFlye spot cold-gaps \
    --bcf /Poppy/mfedarko/sftests/tutorial-output/p15-fdr1pct.bcf \
    --output-coldspots /Poppy/mfedarko/sftests/tutorial-output/coldspot-gaps-minlen5000-nocircular.tsv

--------
strainFlye spot cold-gaps @ 0.00 sec: Starting...
Input BCF file: /Poppy/mfedarko/sftests/tutorial-output/p15-fdr1pct.bcf
Input minimum coldspot gap length: 5000
Check for circular coldspot gaps?: No
Output file describing coldspot gaps: /Poppy/mfedarko/sftests/tutorial-output/coldspot-gaps-minlen5000-nocircular.tsv
--------
strainFlye spot cold-gaps @ 0.00 sec: Loading and checking the BCF file...
strainFlye spot cold-gaps @ 13.76 sec: Looks good so far.
--------
strainFlye spot cold-gaps @ 13.76 sec: Going through contigs and identifying coldspot gaps...
strainFlye spot cold-gaps @ 17.33 sec: Identified 15,258 coldspot gap(s) across all 468 contigs in the BCF file.
--------
strainFlye spot cold-gaps @ 17.33 sec: Writing out this information to a TSV file...
strainFlye spot cold-gaps @ 17.42 sec: Done.
--------
strainFlye spot cold-gaps @ 17.42 sec: Done.


Again, `cold-gaps` will output a simple TSV file describing its identified coldspots:

In [9]:
coldspots = pd.read_csv("/Poppy/mfedarko/sftests/tutorial-output/coldspot-gaps-minlen5000-nocircular.tsv", sep="\t")
coldspots.head()

,Contig,Start_1IndexedInclusive,End_1IndexedInclusive,Length
0,edge_8,383672,681281,297610
1,edge_8,681283,715603,34321
2,edge_8,717621,865477,147857
3,edge_8,885778,906394,20617
4,edge_8,913216,920714,7499


## 4. Phasing analyses

### 4.1. Generating "smoothed haplotypes"

Given our called mutations, we can attempt to generate haplotypes that respect these mutations using strainFlye's `smooth` module.

The details and motivation for this are explained in depth in our paper. To briefly summarize, we will convert the reads aligned to each contig into "smoothed reads," which only contain our called mutations with no other variations. We will then (optionally, depending on the `--virtual-reads` parameter) construct "virtual reads" to fill in low-coverage regions. We will then assemble these reads using [LJA](https://github.com/AntonBankevich/LJA/) to construct "smoothed haplotypes."

In [10]:
!strainFlye smooth

Usage: strainFlye smooth [OPTIONS] COMMAND [ARGS]...

  [+] Create and assemble smoothed and virtual reads.

Options:
  -h, --help  Show this message and exit.

Commands:
  create    Create smoothed and virtual reads for each contig.
  assemble  Assemble contigs' reads using LJA.


#### 4.1.1. Create smoothed and virtual reads

In [15]:
!strainFlye smooth create

Usage: strainFlye smooth create [OPTIONS]

  Create smoothed and virtual reads for each contig.

Options:
  -c, --contigs PATH              FASTA file of contigs.  [required]
  --bam PATH                      Sorted and indexed BAM file representing an
                                  alignment of reads to contigs.  [required]
  --bcf PATH                      Indexed BCF file describing single-
                                  nucleotide mutations in a set of contigs.
                                  [required]
  -di, --diversity-indices PATH   TSV file describing the diversity indices of
                                  a set of contigs, produced by one of the
                                  "strainFlye call" commands. Only used if
                                  --virtual-reads is specified. Along with
                                  diversity indices, these files list contigs'
                                  average coverages. If --virtual-reads is
                     

In [ ]:
!strainFlye smooth create \
    --contigs /Poppy/mfedarko/sftests/tutorial-output/sheepgut_contigs_atleast_1Mbp.fasta \
    --bam /Poppy/mfedarko/sheepgut/main-workflow/output/fully-filtered-and-sorted-aln.bam \
    --bcf /Poppy/mfedarko/sftests/tutorial-output/p15-fdr1pct.bcf \
    --diversity-indices /Poppy/mfedarko/sftests/tutorial-output/call-p15/diversity-indices.tsv \
    --output-dir /Poppy/mfedarko/sftests/tutorial-output/smooth/reads

#### 4.1.2. Assemble these reads using LJA

This step assumes that we have already installed [LJA](https://github.com/AntonBankevich/LJA/), in particular the `simple_ec` branch of it. Please see [LJA's manual](https://github.com/AntonBankevich/LJA/blob/main/docs/lja_manual.md) for installation instructions.

I have installed LJA into a specific location on our cluster. So that strainFlye can easily run LJA for each contig's reads files, we pass the location of LJA's binary executable to strainFlye below using the `--lja-bin` parameter.

In [14]:
!strainFlye smooth assemble

Usage: strainFlye smooth assemble [OPTIONS]

  Assemble contigs' reads using LJA.

  Please note that this command relies on the "simple_ec" branch of LJA being
  installed on your system. See strainFlye's README (and/or LJA's manual) for
  details on installing LJA.

Options:
  -r, --reads-dir DIRECTORY   Directory produced by "strainFlye smooth create"
                              containing smoothed (and optionally virtual)
                              reads. We will use LJA to assemble each
                              *.fasta.gz file in this directory (representing
                              reads from different contigs) independently.
                              [required]
  -p, --lja-params TEXT       Additional parameters to pass to LJA, besides
                              the --reads and --output-dir parameters. To
                              explain our defaults: the --simpleec flag is
                              currently only available on the simple_ec branch


In [ ]:
!strainFlye smooth assemble \
    --reads-dir /Poppy/mfedarko/sftests/tutorial-output/smooth/reads \
    --lja-bin /home/mfedarko/software/LJA-branch/bin/lja \
    --output-dir /Poppy/mfedarko/sftests/tutorial-output/smooth/assemblies

### 4.2. Constructing link graphs

(TODO)